# INF8111 - Data Mining


## TP2 Autumn 2019 - Extraction and analysis of a tweets database

##### Team members:

    - Nom (Matricule) 1
    - Nom (Matricule) 2
    - Nom (Matricule) 3

## Overview


In 2017, Twitter had 313 million active users per month with 500 million tweets sent per day. This information is made available to web research and development through a public API that collects the information you want.

Nevertheless, Twitter's development policy limits the sharing of this data. Indeed, sharing the content of tweets in a database is not allowed, only the tweets identifiers are. To publicly share a tweets database that has been created, it is necessary that this database be made up only of tweets identifiers, and this is what is found in most public data sets.

It is therefore necessary to "hydrate" the tweets in question, i.e. extract all the information from the ID, which requires using the Twitter API.

We will use here public databases created by GWU (George Washington University), which have the advantage of being very recent:
https://dataverse.harvard.edu/dataverse/gwu-libraries

Each GWU database covers a specific topic (2016 U.S. election, Olympic Games, etc.), and the data was collected by applying queries that filtered the results to have only relevant tweets. A README file is provided with each database to give details of the creation of the *dataset*. 

**Thus, the objectives of this TP are as follows:**

 1. Build a crawler that collects tweet information from its ID, with the dataset of its choice and the relevant information for the chosen subject
 2. From these collected Twitter data, application of Machine Learning (ML)/Natural Language Processing (NLP) methods to provide relevant analysis. 

Twitter allowing the **local** sharing of data (for example within a research group), a database will be provided if you are unable to create your own.

# I/ Hydration of tweets using the Twitter API (4 Pts)

### 1. Getting Twitter authorization to use the API

For authentication, Twitter uses OAuth : https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
You will need OAuth2 in particular here, because you will not interact with users on Twitter (simply collected data).

##### 1.1. Obtaining a developper account

 The first step required to register your application and create a Twitter developer account. To do this:

 - Create a basic Twitter account (if you don't have one)
 
 - On the website https://developer.twitter.com, click on *apply* to obtain a developer account. 
 
 - Fill in all the necessary fields. Twitter asks for a lot of details on how you will use this account, so it is important to explain the approach in detail: it is important to underline the fact that the project is **academic** (no commercial intention, no publication of the data collected, etc.), explain the objectives and learning of this TP (familiarisation with the Twitter API, the concrete application of Data Mining methods, etc.), but also explain in detail what you will do with the data, the methods you will apply, the report provided, etc.  If you are not specific enough, Twitter can send you an email to ask for clarification. 


##### 1.2. Obtaining access tokens 

 - When Twitter has validated your developer account request, go to https://developer.twitter.com/en/apps to create an app.

- Again, information is to be provided here. Some, like the name or the website, are not very important, you can put a fake website if you want.

- At the end of this process, you can finally get the keys and tokens to use the API: go to the application page to create the tokens. You must retrieve a key pair and a pair of tokens to move on.


In [ ]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""

oauth_token = ""
oauth_secret = ""

###  2. First steps with Twython

##### 2.1 Installation and import of the library


Several Python libraries exist to manipulate the Twitter API. Also called wrappers, they are a set of python functions that call API functions. Among them, we will use Twython, a widespread and actively maintained library.

Twython documentation : https://twython.readthedocs.io/en/latest/api.html 

In [ ]:
import csv
import time
import sys

try:
    from twython import Twython, TwythonError, TwythonRateLimitError
except ImportError:
    !pip install twython

##### 2.2 Creation of an app and first test:

In [ ]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, oauth_token, oauth_secret)

Here is a test with a very simple search to make sure that the request works.

The search function returns a (non-exhaustive) search for tweets, and the "popular" option returns the most popular results of the answer. (documentation here: https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)

In [ ]:
basic_search = twitter.search(q='python', result_type='popular')

The search function returns a dictionary containing the tweets list of the request, and the metadata.

Here is an example of a search result, so observe all the data/metadata that a tweet contains and that you can extract later:

In [ ]:
basic_search['statuses'][0]

It is also possible with Twython to retrieve the information of a tweet from its ID.

#### Question 1. Display the date, username and content of the tweet with the ID : 1157345692517634049

*Hint : you can use the twython function `show_status`*

In [ ]:
test_id = "1157345692517634049"



**Warning:** Twitter has a 15-minute window request limit, which is therefore to be taken into account in the database:  https://developer.twitter.com/en/docs/basics/rate-limiting.html

### 3. Hydratation of a tweets database

The serious business is starting!

We now want to build a `hydrate_database` function that, from a text file containing a list of tweets IDs, creates a csv file containing the information we want to extract.

Due to the request limitation, the `show_status` function seen above is not very effective for this task: at 900 requests for 15 minutes, it will take far too long to build an interesting database. The `lookup_status` function (see documentation) will therefore be more suitable. It will allow to hydrate 100 tweets per request, which, with a limit of 900 requests for 15 minutes, makes the construction of the database more realistic. It will still be necessary to manage the error generated by the limitation, if you want to have more than 90000 tweets or if you call the function several times in less than 15 minutes.

#### Question 2. Implement the method `hydrate_database`

*Warning : It is also necessary to manage the case where the requested feature is not a dictionary key but a subkey, as is the case for the user name for example.*

*Hint : The `sleep` function of time library allows to wait the desire time.*

In [ ]:
def hydrate_database(filename, database_name, 
                     features, nb_requests, 
                     tweet_hydratation_limit=100):
    """
    Create a csv file that contains features of tweets from an file that contains ID of tweets.
    
    filename: Name of the file that contains ids
    database_name: name of the file that will be created
    features: List of features
    nb_requests: number of time the function lookup_status will be called
    tweet_hydratation_limit:
    """
    # Opening the ID File:
    file = open(filename, "r")
    
    # Creation of the file that will contain the hydrated tweets:
    with open(database_name, 'w', newline='', encoding="utf-8") as csvfile:
            
            # TODO
            
            try: # If you don't reach the limit of requests

                # TODO
            
            except TwythonError as e:
                if isinstance(e, TwythonRateLimitError):
                    retry_after = int(e.retry_after)
                    
                    # TODO

    file.close()


Use this file as an example : 
https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/5QCCUU/QPYP8G&version=1.1

We only want to keep the content (*text*) and the user ID (*user/screen_name*)

In [ ]:
filename = "gwu/climate_id.txt"
database_name = "databases/climate.csv"
features = [['text'], ['user', 'screen_name']]
nb_requests = 400

hydrate_database(filename, database_name, features, nb_requests, tweet_hydratation_limit=100)

# II/ Analysis of a database of your choice  (16 pts)

Now that you are able to hydrate a tweets database efficiently and taking into account the limitations of Twitter, you can apply it to the *dataset* that interests you most.

### 1. Instructions

In this section, you will conduct a **full** Data Science project, from data collection to interpretation of the results. You must choose from the following 3 topics:
 
 1. Sentiment analysis for prediction of the results of the American election.

    **Dataset**: "2016 United States Presidential Election Tweet Ids", https://doi.org/10.7910/DVN/PDI7IN  
 
    **Note:** This subject is quite similar to TP1 (with here feeling = political party), so you are free to take up what you did. However, we should go a little deeper here, for example on the classification stage. In addition, you have a new problem here which is that your data is not labelled (but the construction of the collections should allow you to label yourself).


 2. Hate speech detection.
    
    **Dataset**: Modify your hyrate function to generate a database with recent tweets, using the `search` function.
 
    **Note:** This subject could also be addressed in the same way as TP1: preprocessing + classification steps. However, in this case, having data with labels "hateful"/ "not hateful" is much more complex, because many databases labeled, when hydrated, will be almost empty, because the tweets will have been deleted by the time we make our request (because Twitter also ensures that hate tweets are deleted). That's why you have to create a database with the most recent tweets possible, before they are potentially deleted. To designate a tweet as hateful, one method would be to detect hateful vocabulary, for example with `hatebase.org`, which offers large and comprehensive databases. You can create an account on the website to access the API, and then use this library for Python: https://github.com/DanielJDufour/hatebase. If you modify the query to have only tweets containing this vocabulary, and mix it with sentiment analysis, you can get results to analyze. You could also have a "user" approach to search for hate tweets: when a tweet is detected as hate, inspect all the user's tweets and/or followers' tweets. To sum up, you have many possibilities, but this subject is probably the most complex one (but I will take it into account). I will therefore be less demanding on 'quantified' results, the important thing here is more the analysis, and having a coherent approach (it is also very important to take the time to think about a clear definition of 'hateful').


 
 3. Clustering methods applied tweets on current events, and analysis of results.

    **Dataset**: "News Outlet Tweet Ids", https://doi.org/10.7910/DVN/2FIFLH

    **Note:** Application of preprocessing methods, then clustering methods to group tweets that mention the same news or news category (your choice!), then visualization, evolution in function of time... You will need to find out what is the best clustering method, and this will depend on your approach (number of classes known? if yes, how many classes?)

You are entirely free on the whole process (choice of extracted information, ML methods, library, etc.). As these topics are popular in the scientific community, you can (**if you wish**) draw inspiration from articles in the literature, provided you quote it in your report and make your own implementation. 


#### The objective here, however, is not to obtain the state of the art, but to apply a clear and rigorous methodology that you have built yourself. 

Since datasets are massive, it is strongly discouraged to make a database containing all hydrated tweets (for example, the authors of database #1 point out that with the limitations of the API it would take you about 32 days). It is up to you to see what size dataset you need.

If you are doing supervised learning: you will need to train a model with a labelled set, and so you have two options available. Either you will have to retrieve labelled data, or you are able to label your data yourself (for example, in the case of subject 1, the database is divided into collections, some of which depend on the political party). You can reuse your TP1 implementation, but you are asked to explore a little more deeply here, especially the classification methods.

Also remember to read the README file corresponding to the database you have chosen, in order to help you better understand your future results.

### 2. Report

For this TP, you will have to provide a report that details and justifies your overall method, and provides the results you have obtained. The following elements must appear (this can be used as a plan, but it is not rigid):

- Project title, and name of all team members (with mail and matricule)
    
- **Introduction** : summary of the problem, methodology and results obtained

- **Dataset presentation** : description, justification of size, choice of features, etc. 

- **Preprocessing** : if any, justification of the preprocessing steps  

- **Methodology** : description and justification of all choices (algorithms, hyper-parameters, regularization, metrics, etc.)

- **Résults** : analysis of the results obtained (use figures to illustrate), linking design choices to the performance obtained.

- **Discussion** : discuss the advantages and disadvantages of your approach; what are the weaknesses, the flaws? What can be improved? You can also suggest future exploration ideas.

- **References** : if you have been inspired by a study already done.
    
You can use the arXiv template for the report : https://fr.overleaf.com/latex/templates/style-and-template-for-preprints-arxiv-bio-arxiv/fxsnsrzpnvwc. **However, the entire report should not exceed 5 pages, including figures and references.** The 5 pages are not mandatory, if you think that less is enough and your report is indeed complete, you will not be penalized.


### 3. Expected files

At the end of the zip file, you will submit a *zip* file with all the following elements:

- The pdf file of the report
- This notebook that you will have completed. You can also implement your method further here, or use another file if you wish (the code must be commented and clear).
- Do not send the data files because they are too large. With the report and the code, everything will be detailed and it will be possible to do it again easily.

### 4. Evalutation

75% of the grade (which is 12 points) of this part will be based on the methodology, and 25% (4 points) on the results.

The rating on the methodology included: 

- The relevance of all the steps of the approach

- The correct description of the selected algorithms

- The judicious justification of the established choices

- A relevant analysis of the results

- Clarity and organization of the report (figures, tables) and code.


As for the results, it is impossible to set a fixed scale because they will depend on the chosen subject. This is a problem you will face: each problem being specific, it can be complicated to qualitatively evaluate a model, especially since you probably do not know the state of the art. That is why it will be important to do several tests, and to compare different methods. Thus, the results must be consistent with the complexity of your implementation: a simple and naive model will provide you with initial results, which you will then have to improve with more precise and complex models.

Therefore, all points for the results will be given if: 
 - You obtain first results with a naive method that testify to the relevance of your choices 
 - These results are then significantly improved with a more complex method
 - All this is well justified and put into the context of the problem 